<a href="https://colab.research.google.com/github/zzzzzuuuuu/big-data-analytics/blob/main/More_on_LS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 선형 모델 학습 심화
## 선형 회귀 학습 심화
- 하이퍼 파라미터 튜닝 (GridSearchCV)
- 데이터 전처리 (Data Processing)
- 파이프라인 (Pipeline)

## 하이퍼 파라미터 튜닝 (GridSearchCV)
### 하이퍼 파라미터 튜닝의 필요
- learning_rate (step size) 및 정규화 정도에 따라 성능 차이가 있음
- 가장 높은 성능으로 학습하게 하는 하이퍼 파라미터를 찾을 필요가 있음
- 예시
  - 라쏘 회귀에서, alpha=[0.0001, 0.001, 0.01, 0.1, 1, 10, 100]의 값 중 어떤 alpha 값이 가장 좋은지?
  - 라쏘 회귀에서, eta0=[0.0001, 0.001, 0.01, 0.1]의 값 중 어떤 eta0 값이 가장 좋은지?
  - 두 경우를 곱하면 28가지의 경우 중 어떤 조합이 가장 좋은지?

검증 데이터 셋 + for문을 활용해서 해결 가능 - #1

In [6]:
from sklearn.datasets import load_diabetes
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np

## 1. 검증 데이터셋 + for문
p.s. 근데 for문 이제.. 안.. 쓴다...

In [7]:
# 가장 좋은 eta, alpha 찾기
l_eta0 = [0.0001, 0.001, 0.01, 0.1]
l_alpha = [0.0001, 0.001, .001, 0.1, 1, 10, 100]

In [8]:
X, y = load_diabetes(return_X_y=True, as_frame=True)
# 학습데이터 : 테스트데이터 = 4 : 1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)
# 학습데이터 중에서 25%만큼 validation으로 쪼갬
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1234)

best_R2=0 # 검증데이터 set에 대한 R2
for eta0 in l_eta0:
    for alpha in l_alpha:
        reg = SGDRegressor(penalty='l2', max_iter=100000, learning_rate='constant', eta0=eta0,alpha=alpha, random_state=1234) # l1 = 라쏘, l2 = 릿쏘
        reg = reg.fit(X_train, y_train)
        current_R2=reg.score(X_val,y_val) # 검증 데이터
        if current_R2>best_R2:
            best_R2=current_R2
            best_eta0=eta0
            best_alpha=alpha

# 새롭게 학습
reg = SGDRegressor(penalty='l2', max_iter=100000, learning_rate='constant', eta0=best_eta0,alpha=best_alpha, random_state=1234)
reg = reg.fit(X_train, y_train)
test_R2=reg.score(X_test,y_test)


print('최적 러닝레이트:',best_eta0)
print('최적 정규화 계수:',best_alpha)
print('검증 데이터셋 R2:',best_R2)
print('테스트 데이터셋 R2:',test_R2)

최적 러닝레이트: 0.0001
최적 정규화 계수: 0.0001
검증 데이터셋 R2: 0.5349586246461101
테스트 데이터셋 R2: 0.4731219201091438


**## 하이퍼 파라미터 튜닝 (GridSearchCV)** (완전개대박중요)
### 하이퍼 파라미터 튜닝의 필요
- 검증 데이터 셋이 어떻게 선택되느냐에 따라 최적의 파라미터가 달라질 수 있음 *p.5
  - k-겹 교차 검증(K-Fold Cross Validation) 활용
    - 데이터를 여러 조각(k개)으로 나눠서 여러번 학습-검증을 반복하는 것
    - 이를 통해 데이터셋의 분할 방식(어떤 데이터를 훈련/검증으로 나누느냐)에 따라 성능이 변하지 않도록 안정적인 결과 제공
  - 검증 데이터 셋을 여러 조합으로 평가한 후 스코어를 평균

### GridSearchCV() - #2
  - 모델이 잘 작동하도록 최적의 설정값(하이퍼파라미터)을 찾아주는 도구
  - 다양한 하이퍼 파라미터 조합을 테스트하고, 각 조합에 대해 교차 검증을 수행한 뒤, 가장 좋은 성능을 내는 하이퍼파라미터를 찾음

  - GridSearchCV 객체 생성
    - 해당 객체로 fit을 할 시에, k-겹 교차검증 + for문을 통한 반복 실험으로 최적의 파라미터 조합을 찾아서 fit
  - for문을 통해 grid에서 cross-validation(CV)을 통해 최적의 파라미터 조합을 찾기 때문에 GridSearchCV

  - sklearn.model_selection에서 import
  - 교차검증을 위해서 KFold 함수를 통해 validator 객체 생성
  - 해당 객체로 fit 적용시, CV를 통해 최적의 파라미터로 fit
    - validation의 경우 훈련 중에만 사용하여, 하이퍼파라미터를 최적화하기 위해 성능을 측정하는데 사용
  - 학습 모델 객체처럼 아래 method 적용 가능
    - fit
    - score
    - predict
  - best_params_: 가장 좋은 조합의 하이퍼 파라미터 반환
  - best_score_: 가장 좋은 조합의 스코어 반환

## 2. GridSearchCV **중요**

In [9]:
from sklearn.model_selection import KFold, GridSearchCV, cross_val_score

X, y = load_diabetes(return_X_y=True, as_frame=True)
# train, test 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

# {튜닝하고자하는 파라미터 이름: 어떤 후보에서 최적의 파라미터를 찾을지 그 후보군}
param_grid = {'alpha': l_alpha, 'eta0': l_eta0}
kfold = KFold(n_splits=4, shuffle=True, random_state=1234) # fold 개수는 n_splits에 적음

# estimator에는 GridSearchCV가 하이퍼파라미터를 튜닝할 수행 대상 모델을 넣음
# SGDRegressor은 확률적 경사하강법 회귀 문제 해결 모델로, 하이퍼파라미터에 따라 성능이 크게 달라질 수 있는 모델
grid_search = GridSearchCV(estimator=SGDRegressor(penalty='l2', max_iter=100000,
                           learning_rate='constant', random_state=1234),
                           param_grid=param_grid, cv=kfold)
grid_search.fit(X_train, y_train) # 28가지 조합 중 4fold cross validation을 했을 때 가장 좋음

y_pred_test = grid_search.predict(X_test)
test_MAE = np.abs(y_pred_test - y_test).mean()

print('최적 러닝레이트:',grid_search.best_params_['eta0']) # 최적의 eta0 리턴받음
print('최적 정규화 계수:',grid_search.best_params_['alpha']) # 최적의 alpha 리턴받음
print('최적 모델의 교차검증 스코어', grid_search.best_score_) # 최적의 eta0 + alpha 조합일 때에 score
print('테스트 데이터셋 R2:',grid_search.score(X_test,y_test)) # test data set을 통해 score 낼 수 있음, 회귀 객체이기 때문에 R2 score가 나옴

최적 러닝레이트: 0.0001
최적 정규화 계수: 0.0001
최적 모델의 교차검증 스코어 0.4859305335296367
테스트 데이터셋 R2: 0.4723402669032044


### tips data에서 tip을 나머지 피쳐로 릿지 회귀하는 문제에서, 최적의 파라미터 찾기 (4-겹 교차검증을 통해 l_alpha, l_eta0 중 최적의 alpha, eta0 찾기)

In [10]:
import pandas as pd

tips = pd.read_csv("./tips.csv")
tips.head()
tips['smoker']=tips['smoker'].map({'No':0,"Yes":1})
tips['day']=tips['day'].map({'Thur':0,"Fri":1,"Sat":2,"Sun":3})
tips['time']=tips['time'].map({'Lunch':0,"Dinner":1})
X_train, X_test, y_train, y_test = train_test_split(tips.drop(columns="tip"),tips.tip,test_size=0.2,random_state=1234)

In [11]:
l_eta0=[0.0001, 0.001, 0.01, 0.1]
l_alpha=[0.0001, 0.001, 0.01, 0.1 ,1, 10, 100]

In [12]:
param_grid = {'alpha': l_alpha, 'eta0': l_eta0}
kfold = KFold(n_splits=4, shuffle=True, random_state=1234)
grid_search = GridSearchCV(estimator=SGDRegressor(penalty='l2', max_iter=100000, learning_rate='constant', random_state=1234),
                           param_grid=param_grid, cv=kfold)
grid_search.fit(X_train, y_train)

print(f'최적 러닝레이트: ', grid_search.best_params_['eta0']) # 최적의 eta0
print(f'최적 정규화 계수: ', grid_search.best_params_['alpha']) # 최적의 alpha

최적 러닝레이트:  0.0001
최적 정규화 계수:  0.0001


## 데이터 전처리
- 결측치 처리
- 스케일링
- 범주형 데이터 처리

-> 사이킷런에서 처리 객체 제공
ex. StandardScaler(), MinMaxScaler(), SimpleImputer(), LabelEncoder(), OneHotEncoder()

- 데이터 병합
- 피쳐 선택

=> 처리 객체를 파이프라인처럼 쌓아서 전처리 및 선형모델을 하나의 파이프라인으로 함수화할 수 있음
이 측면에서, 처리 객체를 활용하는 것이 큰 이점을 가짐

### 결측치 처리
- dropna, fillna 등의 판다스에서 제공되는 메소드 활용
- 그룹연산을 통해서, 그룹별로 결측치를 처리할 수 있음

### 결측치 처리 객체
- from sklearn.impute import SimpleImputer
- imputer=SimpleImputer(**missing_values**=np.nan, **strategy**='mean')
- missing_values는 어떤 값을 결측치로 여길 것인지,
- strategy는 어떤 전략으로 채울 것인지를 의미

- imputer = SimpleImputer() - #3
  - imputer.fit(X)
    - X를 통해 변환기가 fit을 수행 (예시: 평균값을 fit)
  - imputer.transform(X)
    - X의 결측치를 fit한 결과를 통해 채움
  - imputer.fit_transform(X)
    - X를 통해 fit과 transform을 동시에 수행


## 3. 결측치 처리 객체 (SimpleImputer)

In [13]:
from sklearn.impute import SimpleImputer

X, y = load_diabetes(return_X_y=True, as_frame=True)
# 랜덤한 10개의 행(0~데이터프레임X의행개수 사이에서 랜덤으로 10개의 행 인덱스 생성)
# 랜덤한 3개의 열(0~데이터프레임X의열개수 사이에서 랜덤으로 3개의 열 인덱스 생성)을 선택하여 해당 위치 NaN로 채움
X.iloc[np.random.randint(0, X.shape[0], 10), np.random.randint(0, X.shape[1], 3)] = np.nan
X.isna().sum()

,0
age,0
sex,0
bmi,0
bp,0
s1,0
s2,0
s3,10
s4,10
s5,10
s6,0


In [14]:
X = X.fillna(X.mean())
X.isna().sum()

,0
age,0
sex,0
bmi,0
bp,0
s1,0
s2,0
s3,0
s4,0
s5,0
s6,0


In [15]:
X, y = load_diabetes(return_X_y=True, as_frame=True)
X.iloc[np.random.randint(0, X.shape[0], 10), np.random.randint(0, X.shape[1], 3)] = np.nan
X.isna().sum()

,0
age,0
sex,0
bmi,0
bp,0
s1,10
s2,0
s3,10
s4,0
s5,10
s6,0


In [16]:
# 데이터프레임 X의 각 열의 평균값을 계산하여 반환 -> 시리즈 형태
X_mean = X.mean()

In [17]:
# 결측값을 대체하기 위한 도구 SimpleImputer
imputer = SimpleImputer()
imputer.fit(X) # X를 분석하여 결측치를 어떻게 채울지 학습 ...
X = imputer.transform(X) # X의 평균값 계산 .. ???  반환값은 Numpy 배열...
np.sum(X==np.nan) # NaN인지 확인.. 근데 항상 False 반환해서 np.isnan(X)가 맞다는데?

0

In [18]:
Y = np.full_like(X, np.nan) # 모든 값이 다 결측치
# Y는 모두 NaN로 되어있으므로, imputer는 fit에서 학습한 각 열의 평균값으로 모든 값을 채움
Y = imputer.transform(Y) # 모든 값이 다 평균값
Y - np.array(X_mean)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

## 데이터 전처리
### 스케일리 처리 객체
- MinMaxScaler()(최소, 최대) 혹은 StandardScaler()(평균, 표준편차)를 통해 객체 생성 - #4
  - scaler.fit(X)
    - X를 통해 변환기가 fit을 수행 ex. 최대/최소, 평균/표준편차 fit
  - scaler.transform(X)
    - fit한 결과를 통해 스케일링 진행
  _ scaler.fit_transform(X)
    - X를 통해 fit과 transform을 동시에 수행


## 4. 스케일링

In [19]:
import numpy as np
from sklearn.model_selection import train_test_split

X, y = np.arange(120).reshape((30, 4)), list(range(30))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1234)

In [20]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train) # transform 진행하기 위해 fit을 통해 주어진 평균, 분산 계산

X_test_scaled = scaler.transform(X_test) # 스케일링
X_test_scaled[:5, :]

array([[-1.10387637, -1.10387637, -1.10387637, -1.10387637],
       [-0.74391669, -0.74391669, -0.74391669, -0.74391669],
       [-1.46383606, -1.46383606, -1.46383606, -1.46383606],
       [-1.82379575, -1.82379575, -1.82379575, -1.82379575],
       [ 1.41584143,  1.41584143,  1.41584143,  1.41584143]])

In [22]:
X, y = np.arange(120).reshape((30, 4)), list(range(30))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1234)
scaler.fit_transform(X_train) # X를 통해 fit과 transform 동시에 수행

array([[ 0.69592206,  0.69592206,  0.69592206,  0.69592206],
       [ 0.93589519,  0.93589519,  0.93589519,  0.93589519],
       [-1.94378231, -1.94378231, -1.94378231, -1.94378231],
       [-1.70380918, -1.70380918, -1.70380918, -1.70380918],
       [ 1.29585487,  1.29585487,  1.29585487,  1.29585487],
       [ 1.17586831,  1.17586831,  1.17586831,  1.17586831],
       [ 0.21597581,  0.21597581,  0.21597581,  0.21597581],
       [-1.3438495 , -1.3438495 , -1.3438495 , -1.3438495 ],
       [-0.02399731, -0.02399731, -0.02399731, -0.02399731],
       [ 1.05588175,  1.05588175,  1.05588175,  1.05588175],
       [-0.62393012, -0.62393012, -0.62393012, -0.62393012],
       [-0.86390325, -0.86390325, -0.86390325, -0.86390325],
       [ 0.09598925,  0.09598925,  0.09598925,  0.09598925],
       [ 1.535828  ,  1.535828  ,  1.535828  ,  1.535828  ],
       [ 0.45594894,  0.45594894,  0.45594894,  0.45594894],
       [-0.50394356, -0.50394356, -0.50394356, -0.50394356],
       [ 0.5759355 ,  0.

### 범주형 데이터 처리
- LabelEncoder - #5
  - 범주형 데이터를 정수로 변환 (numeric type)
  - encoder.fit(X)
    - X를 통해 범주형데이터를 정수와 매칭 (다차원 DF X, 각 feature)
  - encoder.transform(X)
    - fit된 매칭관계를 통해 X 변환
  - encoder.inverse_transform(X)
    - 정수를 원래의 범주형 데이터로 반환

- OneHotEncoder - #6
  - 범주형 데이터를 더미변수로 변환(0과 1로 이루어짐)
    - ex (1,0,0), (0,0,1)
  - encoder.fit(X)
    - X를 통해 범주형데이터를 정수와 매칭
  - encoder.transform(X)
    - fit된 매칭관계를 통해 X를 변환
  - encoder.inverse_transform(X)
    - 정수를 원래의 범주형 데이터로 변환

## 5. LaeblEncoder

In [32]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

tips = pd.read_csv("./tips.csv")

# 그냥 하나하나 다 하는.. 방식
tips['smoker']=tips['smoker'].map({'No':0,"Yes":1})
tips['day']=tips['day'].map({'Thur':0,"Fri":1,"Sat":2,"Sun":3})
tips['time']=tips['time'].map({'Lunch':0,"Dinner":1})
tips.head()

,total_bill,tip,smoker,day,time,size
0,16.99,1.01,0,3,1,2
1,10.34,1.66,0,3,1,3
2,21.01,3.50,0,3,1,3
3,23.68,3.31,0,3,1,2
4,24.59,3.61,0,3,1,4


In [33]:
tips = pd.read_csv("./tips.csv")

# LabelEncoder 사용
encoder = LabelEncoder()
encoder.fit(tips.smoker)
tips.smoker = encoder.transform(tips.smoker)
tips.head()

,total_bill,tip,smoker,day,time,size
0,16.99,1.01,0,Sun,Dinner,2
1,10.34,1.66,0,Sun,Dinner,3
2,21.01,3.50,0,Sun,Dinner,3
3,23.68,3.31,0,Sun,Dinner,2
4,24.59,3.61,0,Sun,Dinner,4


In [34]:
encoder.classes_

array(['No', 'Yes'], dtype=object)

In [36]:
encoder.inverse_transform([1,1,1,1,1,1,1])

array(['Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes'], dtype=object)

In [38]:
encoder2 = LabelEncoder()
encoder2.fit(tips.day) # 범주형 데이터를 정수와 매칭
tips.day = encoder2.transform(tips.day) # fit된 매칭관계를 통해 X 변환
tips.head()

,total_bill,tip,smoker,day,time,size
0,16.99,1.01,0,2,Dinner,2
1,10.34,1.66,0,2,Dinner,3
2,21.01,3.50,0,2,Dinner,3
3,23.68,3.31,0,2,Dinner,2
4,24.59,3.61,0,2,Dinner,4


In [39]:
encoder2.classes_

array(['Fri', 'Sat', 'Sun', 'Thur'], dtype=object)

In [40]:
encoder3 = LabelEncoder()
encoder3.fit(tips.time)
tips.time = encoder3.transform(tips.time)
tips.head()

,total_bill,tip,smoker,day,time,size
0,16.99,1.01,0,2,0,2
1,10.34,1.66,0,2,0,3
2,21.01,3.50,0,2,0,3
3,23.68,3.31,0,2,0,2
4,24.59,3.61,0,2,0,4


In [41]:
encoder3.classes_

array(['Dinner', 'Lunch'], dtype=object)

## 6. OneHotEncoder

In [42]:
import pandas as pd

tips = pd.read_csv("./tips.csv")
tips.head()

,total_bill,tip,smoker,day,time,size
0,16.99,1.01,No,Sun,Dinner,2
1,10.34,1.66,No,Sun,Dinner,3
2,21.01,3.50,No,Sun,Dinner,3
3,23.68,3.31,No,Sun,Dinner,2
4,24.59,3.61,No,Sun,Dinner,4


In [44]:
# 기존 get_dummies() 함수
pd.get_dummies(tips.day, dtype='int')

,Fri,Sat,Sun,Thur
0,0,0,1,0
1,0,0,1,0
2,0,0,1,0
3,0,0,1,0
4,0,0,1,0
...,...,...,...,...
239,0,1,0,0
240,0,1,0,0
241,0,1,0,0
242,0,1,0,0


In [50]:
# sparse_output을 False로 해야 원하는 대로 얻을 수 있음. True로 하면 matrix꼴로 받는다..
encoder = OneHotEncoder(sparse_output=False)
# Series의 value만 갖고, reshape 해줘야 함 (1차원 배열 -> 2차원 배열)
encoder.fit(tips.day.values.reshape(-1,1))
encoder.transform(tips.day.values.reshape(-1,1))[:5,:]

array([[0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.]])

In [53]:
encoder.inverse_transform([[1,0,0,0],[0,1,0,0]])

array([['Fri'],
       ['Sat']], dtype=object)

## 파이프라인 - #7
- 변환기를 쌓아서, 하나의 함수처럼 만들 수 있게 하는 사이킷런의 유용한 클래스
- 마지막에는 regression 객체처럼 prediction을 할 수 있는 학습 모델 객체가 포함되어야 함 (마지막)
- 파이프라인을 활용하지 않는 경우 코드가 굉장히 길어지며, 복잡해진다!!

- from sklearn.pipeline import Pipeline
- pipeline = Pipeline(steps=[('imputer', SimpleImputer()), ('scaler', StandardScaler()), ('basemodel', SGDRegressor())])
  - steps 옵션에 튜플의 리스트형태로 변환기 및 모델 객체를 순서대로 입력
  - 이름과 객체를 순서대로 작성

- pipeline.fit(X,y)
  - 각 변환기 별로 순서대로 fit과 transform이 이루어지면서 fit
- pipeline.predict(X)
  - X를 순서대로 transform하여 y_pred를 반환
- pipeline.score(X,y)
  - y_pred를 계산하여 score 반환

## 7. 파이프라인 적용

In [54]:
### tips 데이터 파이프라인 적용

In [55]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

In [58]:
tips = pd.read_csv("./tips.csv")
# 연속형 이산화 (중요)
tips.tip = pd.cut(tips["tip"], bins=3, labels=["low", "middle", "high"])
tips["smoker"] = tips["smoker"].map({"No":0, "Yes":1})
tips["day"] = tips["day"].map({"Thur":0, "Fri":1, "Sat":2, "Sun":3})
tips["time"] = tips["time"].map({"Lunch":0, "Dinner":1})

X_train, X_test, y_train, y_test = train_test_split(tips.drop(columns="tip"), tips["tip"], test_size=0.2, random_state=1234)

tips.head()

,total_bill,tip,smoker,day,time,size
0,16.99,low,0,3,1,2
1,10.34,low,0,3,1,3
2,21.01,low,0,3,1,3
3,23.68,low,0,3,1,2
4,24.59,low,0,3,1,4


In [61]:
tips.iloc[0, 0] = np.nan
tips.iloc[1, 0] = np.nan
tips.iloc[2, 2] = np.nan
tips.iloc[3, 4] = np.nan
tips.iloc[4, 3] = np.nan
tips.iloc[5, 1] = np.nan
tips.iloc[5, 2] = np.nan

tips.head()

,total_bill,tip,smoker,day,time,size
0,NaN,low,0.0,3.0,1.0,2
1,NaN,low,0.0,3.0,1.0,3
2,21.01,low,NaN,3.0,1.0,3
3,23.68,low,0.0,3.0,NaN,2
4,24.59,low,0.0,NaN,1.0,4


In [64]:
pipeline = Pipeline(steps=[('imputer', SimpleImputer()), ('scaler', StandardScaler()),
                           ('basemodel', LogisticRegression(penalty='l2', random_state=1234, max_iter=10000, solver='sag'))])
pipeline.fit(X_train, y_train)
y_pred_test = pipeline.predict(X_test)

print(f'테스트 데이터셋 스코어(정확도): {pipeline.score(X_test, y_test)}')

테스트 데이터셋 스코어(정확도): 0.8571428571428571


In [65]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

print(classification_report(y_test, y_pred_test))

              precision    recall  f1-score   support

        high       0.00      0.00      0.00         1
         low       0.88      0.97      0.93        38
      middle       0.71      0.50      0.59        10

    accuracy                           0.86        49
   macro avg       0.53      0.49      0.50        49
weighted avg       0.83      0.86      0.84        49



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


### 파이프라인 - #8
- **파이프라인과 GridSearchCV를 함께 활용 가능**
- grid_search = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=kfold)
- param_grid={basemodle__C:[0.1,1,10,100,1000]}
  - param_grid에서 변수 이름을 작성 시, pipeline에서 회귀 모델 이름을 적고, "_" 두 개를 적고, 변수 이름을 적기
- grid_search.fit(X,y)
  - 최적의 하이퍼파라미터로 pipeline을 fit
- grid_search.predict(X)
  - 파이프라인을 따라 X를 순서대로 transform하여 y_pred 반환
_ grid_search.score(X,y)
  - 파이프라인을 따라 y_pred를 계산하여 score 반환


## 8. 파이프라인 + 교차검증

### breast_cancer 로지스틱 회귀 (데이터 전처리 + 하이퍼파라미터 튜닝 파이프라이닝)

In [70]:
from sklearn.datasets import load_breast_cancer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

In [71]:
l_C = [0.0001, 0.001, 0.01, 0.1, 1, 10, 100] # 정규화(C)가 커지면 커질수록 정규화 덜함

In [75]:
X, y = load_breast_cancer(return_X_y=True, as_frame=True)
X.iloc[0, 0] = np.nan
X.iloc[1, 0] = np.nan
X.iloc[2, 2] = np.nan
X.iloc[3, 4] = np.nan
X.iloc[4, 3] = np.nan
X.iloc[5, 1] = np.nan
X.iloc[5, 2] = np.nan

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

# 파이프라인 객체 생성
pipeline = Pipeline(steps=[('imputer', SimpleImputer()), ('scaler', StandardScaler()),
 ('basemodel', LogisticRegression(penalty='l2', random_state=1234, max_iter=10000, solver='sag'))])

param_grid = {'basemodel__C': l_C}
kfold = KFold(n_splits=4, shuffle=True, random_state=1234)

In [77]:
# 하이퍼파라미터 튜닝을 위해 로지스틱 회귀 객체 대신 pipeline 객체를 넣음. param_grid 중요
# 결측치처리, 표준화, 모델학습 이 모든 과정을 포함하는 pipeline 객체를 estimator에 넣음
grid_search = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=kfold)
grid_search.fit(X_train, y_train)
y_pred_test = grid_search.predict(X_test)

print('최적 정규화 계수: ', grid_search.best_params_['basemodel__C'])
print(f'최적 모델의 교차검증 스코어(정확도): {grid_search.best_score_}') # 로지스틱 객체이므로 정확도 (분류))
print(f'테스트 데이터셋 스코어(정확도): {grid_search.score(X_test, y_test)}')

최적 정규화 계수:  0.1
최적 모델의 교차검증 스코어(정확도): 0.9780701754385964
테스트 데이터셋 스코어(정확도): 0.9385964912280702


In [79]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

y_pred_test = grid_search.predict(X_test)
print(classification_report(y_test, y_pred_test))

              precision    recall  f1-score   support

           0       1.00      0.84      0.92        45
           1       0.91      1.00      0.95        69

    accuracy                           0.94       114
   macro avg       0.95      0.92      0.93       114
weighted avg       0.94      0.94      0.94       114



### diabetes 데이터 릿지 회귀

In [80]:
from sklearn.datasets import load_diabetes
from sklearn.linear_model import SGDRegressor
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler

In [88]:
X, y = load_diabetes(return_X_y=True, as_frame=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

X.iloc[0, 0] = np.nan
X.iloc[1, 0] = np.nan
X.iloc[2, 2] = np.nan
X.iloc[3, 4] = np.nan
X.iloc[4, 3] = np.nan
X.iloc[5, 1] = np.nan
X.iloc[5, 2] = np.nan

# 튜닝할 하이퍼파라미터
l_eta0 = [0.0001, 0.001, 0.01, 0.1] # learning rate 초기값
l_alpha = [0.0001, 0.001, 0.01, 0.1, 1, 10, 100] # 정규화 정도

param_grid={'basemodel__alpha': l_alpha, 'basemodel__eta0': l_eta0}
kfold = KFold(n_splits=4, shuffle=True, random_state=1234)

# 릿지회귀이므로 penalty='l2'
pipeline = Pipeline(steps=[('imputer', SimpleImputer()), ('scaler', MinMaxScaler()),
                           ('basemodel', SGDRegressor(penalty='l2', random_state=1234, max_iter=100000))])
grid_search = GridSearchCV(pipeline, param_grid=param_grid, cv=kfold)
grid_search.fit(X_train, y_train)
y_pred_test = grid_search.predict(X_test)

print('최적 러닝레이트: ', grid_search.best_params_['basemodel__eta0'])
print('최적 정규화 계수: ', grid_search.best_params_['basemodel__alpha'])
print('최적 모델의 교차검증 스코어: ', grid_search.best_score_)
print('테스트 데이터셋 R2:', grid_search.score(X_test, y_test))

최적 러닝레이트:  0.1
최적 정규화 계수:  0.001
최적 모델의 교차검증 스코어:  0.4860377578351637
테스트 데이터셋 R2: 0.47554565473809574
